# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [163]:
#Librerias

from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [164]:
#Proporcionamos una URL

URL = "https://ycharts.com/companies/TSLA/revenues"

In [165]:
def DataSeeker(get_url):
    driver = webdriver.Chrome()
    driver.get(get_url)

    #Obtenemos el HTML
    raw_data = driver.find_elements(By.TAG_NAME, "tr")

    #Limpiamos los datos

    data = []
    for i in raw_data:
        data.append(i.text)
        if ("General" in str(i.text)):
            data.remove(i.text)
            break
        elif ("Date" in str(i.text)):
            data.remove(i.text)

    #Cerramos conexion
    driver.quit()
    return data

In [166]:
def SplitData(data):
    #Separamos los valores que queremos ingresar al DataFrame
    split_date = []
    date = []
    str_revenue = []
    flt_revenue = []
    for i in range(len(data)):
        str_revenue.append (data[i].split(" ")[-1])
        split_date.append(data[i].split(" ")[:3])
        date.append(" ".join(split_date[i]))
    
    #Convertir los valores str en float
    for s in str_revenue:
        if ("B" in str(s)):
            s = float(s.replace("B",""))
            flt_revenue.append(s*1000)
        elif ("M" in str(s)):
            s = float(s.replace("M", ""))
            flt_revenue.append(s)

    
    return date, flt_revenue

In [170]:
def TransformDataSet(col1, col2):
    #Creamos el Data Set
    dataset = pd.DataFrame(zip(col1, col2), columns=["Date","Revenue"])

    #Eliminamos las dos primeras filas innecesarias.
    dataset = dataset.drop([0,1,2,3,4,5,6]).reset_index(drop = True)

    return dataset

In [168]:
def SaveIntoDB(dataset):
    # Nos conectamos a la base de datos

    con = sqlite3.connect("tesla.db")

    dataset.to_sql(name=("tesla_quaterly_revenue"), con = con, if_exists="replace", index=False)

    con.commit()
    
    return f"Database is uploaded."



In [169]:
#Ejecutamos las funciones

date, revenue = SplitData(DataSeeker(URL))
SaveIntoDB(TransformDataSet(date, revenue))

'Database is uploaded.'